<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D1%80%D0%BE%D0%B6%D0%B4%D0%B0%D1%8E%D1%89%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Abstract_factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract factory / Абстрактная фабрика
Предоставляет интерфейс для содания семейств взаимосвязанных объектов, не специфицируя их конкретных классов.

abstract_factory.svg

## Абстрактный "интерфейс" продукта фабрики и абстрактная фабрика
Продуктом нашей фабрики будет Лабиринт. 

In [ ]:
from abc import ABC
from random import Random
"""
абстрактный интерфейс для продукта фабрики - лабиринта
"""
class AbstractMaze(ABC):
    def draw(self):
        pass
    def add_room(self, r):
        pass
    def add_door(self,r1,r2):
        pass
"""
абстрактная фабрика для изготовления лабиринтов вне зависимости от их реализации,
главное, чтобы их продукт удовлетворял интерфейсу абстрактного продукта
"""
class MazeFactory (ABC):
    def make_maze(self):
        pass
    def make_room(self):
        pass

## Класс - пользователь фабрики

In [ ]:
"""
класс, использующий абстрактную фабрику лабиринтов. На вход нужно дать только указание о том,
какую конкретную реализацию лабиринтов использовать. Создается лабиринт из 4 комнат и 3 дверей между ними.
"""
class MazeGame:
    def create_maze(self,maze_factory:MazeFactory):
        m = maze_factory.make_maze()
        r1 = maze_factory.make_room()
        r2 = maze_factory.make_room()
        r3 = maze_factory.make_room()
        r4 = maze_factory.make_room()
        m.add_room(r1)
        m.add_room(r2)
        m.add_room(r3)
        m.add_room(r4)

        m.add_door(r1,r2)
        m.add_door(r2,r3)
        m.add_door(r3,r4)

        print(m.draw())
        return m


## Обычный лабиринт - первая конкретная фабрика и конкретный продукт

In [ ]:
"""
класс конкретной фабрики, реализующей интерфейс абстрактной
"""
class SimpleMazeFactory(MazeFactory):
    def make_maze(self):
        return SimpleMaze()
    def make_room(self):
        return SimpleRoom()

"""
класс конкретного продукта NumberMaze соответствует интерфейсу абстрактного продукта
"""
class SimpleMaze(AbstractMaze):
    _data:list
    def __init__(self):
        self._data = list()

    def draw(self):
        res = ''
        for i in self._data:
            if type(i)==SimpleRoom:
               res += ' '+str(i.index)+' '
            else:
                res += str(i)
        return res

    def add_room(self, r):
        self._data.append(r)

    def add_door(self,r1,r2):
        index_r1 = self._data.index(r1)
        index_r2 = self._data.index(r2)
        self._data = self._data[:index_r1+1]+['->']+self._data[index_r2:]

class SimpleRoom():
    all_indexes = []
    index:int
    def __init__(self):
        new_index = 0
        while new_index in SimpleRoom.all_indexes or new_index==0:
            new_index = Random().randint(1,10**6)
        self.index = new_index

## Волшебный лабиринт - вторая конкретная фабрика и конкретный продукт

In [ ]:
"""
класс конкретной фабрики, реализующей интерфейс абстрактной
"""
class EnchantedMazeFactory(MazeFactory):
    def make_maze(self):
        return EnchantedMaze()
    def make_room(self):
        return EnchantedRoom()
"""
класс конкретного продукта, реализующий интерфейс абстрактного
"""
class EnchantedMaze(AbstractMaze):
    _data:list
    
    def __init__(self):
        self._data = list()

    def draw(self):
        res = ''
        for i in self._data:
            if type(i)==EnchantedRoom:
               res += ' '+str(i.magic)+' '
            else:
                res += str(i)
        return res

    def add_room(self, r):
        self._data.append(r)

    def add_door(self,r1,r2):
        index_r1 = self._data.index(r1)
        index_r2 = self._data.index(r2)
        random_direction = Random().randint(0,1)
        left_sign = '<' if random_direction else '-'
        right_sign = '>' if not random_direction else '-' 
        magic_door = left_sign + '-'*Random().randint(1,5) + right_sign
        self._data = self._data[:index_r1+1]+[magic_door]+self._data[index_r2:]

class EnchantedRoom():
    all_indexes = []
    index:int
    magic:str
    def __init__(self):
        new_index = 0
        while new_index in EnchantedRoom.all_indexes or new_index==0:
            new_index = Random().randint(1,10**6)
        self.index = new_index
        self.magic = '*'*Random().randint(1,5)

## Применение

In [ ]:
  # Создаем простой лабиринт
me = MazeGame().create_maze(SimpleMazeFactory())
# me.draw()

 946711 -> 640452 -> 302224 -> 578967 


In [ ]:
# Создаем волшебный лабиринт
ms = MazeGame().create_maze(EnchantedMazeFactory())
# ms.draw()

 ** ----> ** ------> ***** <---- ** 
